<a href="https://colab.research.google.com/github/toussaintma/neuralnetworksfromzerotohero/blob/main/walkthrough_makemore_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# data and code at https://github.com/karpathy/ng-video-lecture
# NanoGPT code at https://github.com/karpathy/nanoGPT
# course at https://www.youtube.com/watch?v=PaCmpygFfXo&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=2&t=11s
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# Letw built GTP: from scratch, in code, spelled out
# GPT 3 paper: https://arxiv.org/pdf/2005.14165.pdf
# Attention is all you need paper: https://arxiv.org/abs/1706.03762


--2023-07-24 12:09:50--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-07-24 12:09:50 (20.4 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [6]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

print('Length of dataset in characters: ', len(text))
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Length of dataset in characters:  1115394


In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab = ''.join(chars)
print(f'Vocabulary of size {vocab_size}: {vocab}')

Vocabulary of size 65: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [8]:
stoi = {s:i for (i, s) in enumerate(chars)}
itos = {i:s for (i, s) in enumerate(chars)}
encode = lambda x: [stoi[a] for a in x]
decode = lambda x: ''.join([itos[a] for a in x])
stoi[itos[13]], decode(encode('ABC'))

(13, 'ABC')

In [9]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [10]:
n = int(0.90 * len(data))
train_data = data[:n]
eval_data = data[n:]

In [11]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f'when input i {context} the target is {target}')

when input i tensor([18]) the target is 47
when input i tensor([18, 47]) the target is 56
when input i tensor([18, 47, 56]) the target is 57
when input i tensor([18, 47, 56, 57]) the target is 58
when input i tensor([18, 47, 56, 57, 58]) the target is 1
when input i tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input i tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input i tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [13]:
torch.manual_seed(1337)

batch_size = 4
block_size = 8

def get_data(split):
  data = train_data if split == 'train' else eval_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i: i+block_size] for i in ix], dim=0)
  y = torch.stack([data[i+1: i+block_size+1] for i in ix], dim=0)
  return x,y

Xb, Yb = get_data('train')

In [152]:
eval_iters = 200
max_iters = 5000
eval_interval = 500
num_heads = 16
n_embedding = 32
n_head = 4
n_layer = 6
dropout = 0.2

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_data(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


### Bigram Model

In [153]:
class Block(nn.Module):
  def __init__(self, n_embedding, n_head):
    super().__init__()
    head_size = n_embedding // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embedding)
    self.ln1 = nn.LayerNorm(n_embedding)
    self.ln2 = nn.LayerNorm(n_embedding)

  def forward(self, x):
    x = self.sa(self.ln1(x)) + x
    x = self.ffwd(self.ln2(x)) + x
    return x

In [154]:
class FeedForward(nn.Module):
  def __init__(self, n_hidden):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_hidden, 4 * n_hidden, bias=False),
        nn.ReLU(),
        nn.Linear(4 * n_hidden, n_hidden, bias=False),
        nn.Dropout(dropout),
    )

  def forward(self, x):
    return self.net(x)

In [155]:
class Head(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.query = nn.Linear(n_embedding, head_size, bias=False)
    self.key = nn.Linear(n_embedding, head_size, bias=False)
    self.value = nn.Linear(n_embedding, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B, T, C = x.shape
    q = self.query(x) # B T head_size
    k = self.key(x) # B T head_size
    wei = q @ k.transpose(-2, -1) * (C ** -0.5) ## (B, T, hs) @ (B, hs, T) -> (B, T, T)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # B T T
    wei = F.softmax(wei, dim=-1) # B T T
    wei = self.dropout(wei)
    v = self.value(x) # B T hs
    out = wei @ v # B T T @ B T hs -> B T hs
    return out

In [156]:
class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(num_heads * head_size, n_embedding)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    return self.dropout(self.proj(out))

In [157]:
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embedding)
    self.position_embedding_table = nn.Embedding(block_size, n_embedding)
    self.blocks = nn.Sequential(*[Block(n_embedding, n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embedding)
    #self.sa_heads = MultiHeadAttention(4, n_embedding // 4) # 4 heads of 8-dimensional self-attention
    #self.ffwd = FeedForward(n_embedding)
    self.lm_head = nn.Linear(n_embedding, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape

    tok_emb = self.token_embedding_table(idx) # Batch * Time * Channel
    pos_emb = self.position_embedding_table(torch.arange(T)) # T C
    x = tok_emb + pos_emb # B T C
    x = self.blocks(x)
    #x = self.sa_heads(x) # B T n_emb
    #x = self.ffwd(x)
    logits = self.lm_head(x) # B T vocab_size

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
      #print(logits)

    return logits, loss

  def generate(self, idx, max_new_token):
    # idx indices of batch B T C because we have no targets
    out = []
    for i in range(max_new_token):
      logits, loss = self(idx[:, -block_size:])
      logits = logits[:, -1,:] # B C
      probs = F.softmax(logits, dim=-1) # B C
      pred = torch.multinomial(probs, num_samples=1) # B 1
      idx = torch.cat((idx, pred), dim=1) # B T+1
    return idx

model = BigramLanguageModel()
logits, loss = model(Xb, Yb)
print(Xb.shape, logits.shape, loss)

idx = model.generate(torch.zeros((1,1), dtype=torch.long), max_new_token=100)[0].tolist()
print(decode(idx))

torch.Size([32, 8]) torch.Size([256, 65]) tensor(4.4605, grad_fn=<NllLossBackward0>)

DM:jRqDtqPLBsIYcPm-Ydzhukb,mQm:YYScDYWMDicgbsIzJWhCYlYzJ!JbQYc .tVotkFp&L!JBNlObOzQFtVccf wwcrdEkrdr


In [158]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [159]:
batch_size = 32

for iter in range(10000):
   # every once in a while evaluate the loss on train and val sets
  if iter % eval_interval == 0 or iter == max_iters - 1:
      losses = estimate_loss()
      print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
  Xb, Yb = get_data('train')
  logits, loss = model(Xb, Yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_token=500)[0].tolist()))


step 0: train loss 4.5462, val loss 4.5541
step 500: train loss 2.4288, val loss 2.4403
step 1000: train loss 2.3095, val loss 2.3271
step 1500: train loss 2.2362, val loss 2.2545
step 2000: train loss 2.1862, val loss 2.2062
step 2500: train loss 2.1598, val loss 2.1938
step 3000: train loss 2.1155, val loss 2.1655
step 3500: train loss 2.1123, val loss 2.1385
step 4000: train loss 2.0930, val loss 2.1391
step 4500: train loss 2.0689, val loss 2.1214
step 4999: train loss 2.0427, val loss 2.0954
step 5000: train loss 2.0495, val loss 2.1072
step 5500: train loss 2.0380, val loss 2.1083
step 6000: train loss 2.0210, val loss 2.0937
step 6500: train loss 2.0230, val loss 2.0817
step 7000: train loss 2.0058, val loss 2.0868
step 7500: train loss 1.9955, val loss 2.0746
step 8000: train loss 1.9881, val loss 2.0564
step 8500: train loss 1.9778, val loss 2.0696
step 9000: train loss 1.9722, val loss 2.0683
step 9500: train loss 1.9706, val loss 2.0417

BRONIUS:

Mirdse of as frounain, good

In [ ]:
#print(decode(model.generate(torch.zeros((1,1), dtype=torch.long), max_new_token=400)[0].tolist()))

### Self-attention trick

In [22]:
torch.manual_seed(1337)

B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [23]:
xbow = torch.zeros((B, T, C))
for b in range(B):
  for t in range(T):
    xprev = x[b, :t+1] # t C
    xbow[b, t] = torch.mean(xprev, 0) # C

print(x[0])
print(xbow[0])
print(x.shape, xbow.shape)

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])
torch.Size([4, 8, 2]) torch.Size([4, 8, 2])


In [24]:
wei = torch.tril(torch.ones(T, T))
wei = wei / torch.sum(wei, dim=1, keepdim=True)
xbow2 = wei @ x
(abs(xbow2 - xbow) < 0.01).all()

tensor(True)

In [25]:
exp = torch.tril(torch.ones(T, T)).transpose(0, 1)
exp = exp / torch.sum(exp, dim=0)
res = torch.Tensor()
for b in range(B):
  xt = x[b].transpose(0, 1)
  r = (xt @ exp).transpose(0, 1)
  #print(xt.shape, exp.shape, r.shape)
  #print(xt)
  #print(exp)
  #print(r)
  res = torch.cat((res, r), dim=0)
res = res.view(4, 8, 2)
print(x.shape, res.shape)
print(x[1])
print(res[1])
print(xbow[1])
(abs(xbow - res) < 0.01).all()

torch.Size([4, 8, 2]) torch.Size([4, 8, 2])
tensor([[ 1.3488, -0.1396],
        [ 0.2858,  0.9651],
        [-2.0371,  0.4931],
        [ 1.4870,  0.5910],
        [ 0.1260, -1.5627],
        [-1.1601, -0.3348],
        [ 0.4478, -0.8016],
        [ 1.5236,  2.5086]])
tensor([[ 1.3488, -0.1396],
        [ 0.8173,  0.4127],
        [-0.1342,  0.4395],
        [ 0.2711,  0.4774],
        [ 0.2421,  0.0694],
        [ 0.0084,  0.0020],
        [ 0.0712, -0.1128],
        [ 0.2527,  0.2149]])
tensor([[ 1.3488, -0.1396],
        [ 0.8173,  0.4127],
        [-0.1342,  0.4395],
        [ 0.2711,  0.4774],
        [ 0.2421,  0.0694],
        [ 0.0084,  0.0020],
        [ 0.0712, -0.1128],
        [ 0.2527,  0.2149]])


tensor(True)

In [26]:
# use softmax()
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
#wei = wei / torch.sum(wei, dim=1, keepdim=True)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


True

### Self-attention

In [27]:
B, T, C = 4, 8, 32
x = torch.rand(B, T, C)

# single head
head_size = 16
query = nn.Linear(C, head_size, bias=False)
key = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # B, T, head_size
q = query(x) # B, T, head_size
wei = q @ k.transpose(-2, -1) * head_size ** -0.5  # B T 16 @ B 16 T = B T T


tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
#out = wei @ x
v = value(x)

out = wei @ v
out.shape

torch.Size([4, 8, 16])

In [29]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * (head_size ** -0.5)
q.var(), k.var(), wei.var()

(tensor(1.0298), tensor(0.8976), tensor(0.9621))